# Asylum seekers

In [1]:
import pandas as pd
import plotly.graph_objects as go
import nbformat

## Analisys

In [2]:
asylum = pd.read_csv('.\\Data\\Clean\\Asylum_data.csv')
population = pd.read_csv(".\\Data\\Clean\\Population_data.csv")

asylum_2000 = asylum[asylum["year"] > 1980]
asylum_2000.head()

,country_of_origin_abbr,country_of_origin_name,country_of_asylum_abbr,country_of_asylum_name,region_of_asylum,category,year,count
1561,AFG,Afghanistan,AUT,Austria,Europe,Asylum-seekers,1981,31
1562,AFG,Afghanistan,FRA,France,Europe,Asylum-seekers,1981,116
1563,AFG,Afghanistan,GBR,United Kingdom,Europe,Asylum-seekers,1981,82
1564,AFG,Afghanistan,DEU,Germany,Europe,Asylum-seekers,1981,3610
1565,AFG,Afghanistan,GRC,Greece,Europe,Asylum-seekers,1981,16


In [3]:
asylum_2000_by_country = asylum_2000.groupby('country_of_origin_name').agg({'count' : 'sum'}).reset_index()
asylum_2000_by_country = asylum_2000_by_country.nlargest(20, 'count')
asylum_2000_by_country.shape


(20, 2)

In [4]:
asylum[asylum['country_of_asylum_name'] == 'Ukraine']

,country_of_origin_abbr,country_of_origin_name,country_of_asylum_abbr,country_of_asylum_name,region_of_asylum,category,year,count
15292,AFG,Afghanistan,UKR,Ukraine,Europe,Refugee,1997,530
16636,SOM,Somalia,UKR,Ukraine,Europe,Refugee,1997,52
16991,AFG,Afghanistan,UKR,Ukraine,Europe,Refugee,1998,447
18305,SOM,Somalia,UKR,Ukraine,Europe,Refugee,1998,22
20451,AFG,Afghanistan,UKR,Ukraine,Europe,Asylum-seekers,2000,662
...,...,...,...,...,...,...,...,...
102136,NGA,Nigeria,UKR,Ukraine,Europe,Asylum-seekers,2024,5
102372,RUS,Russian Federation,UKR,Ukraine,Europe,Asylum-seekers,2024,19
102658,STA,Stateless,UKR,Ukraine,Europe,Asylum-seekers,2024,5
102832,SYR,Syrian Arab Rep.,UKR,Ukraine,Europe,Asylum-seekers,2024,5


# Graph the biggets ex

In [5]:
fig = go.Figure(data = go.Scatter(x = asylum_2000_by_country['country_of_origin_name'], y = asylum_2000_by_country['count'], mode='markers'))
fig.show()

# Adding country population

In [6]:
def Country_population_data(country_code: str, population_data: pd.DataFrame, asylum_data: pd.DataFrame) -> pd.DataFrame:
    country_population = population_data[population_data["Country Code"] == country_code].drop(columns=["Country Name", "Country Code"]).transpose().reset_index()
    country_population.columns = ['year', 'population']
    country_population['year'] = country_population['year'].astype(int)

    country = asylum_data[asylum_data["country_of_origin_abbr"] == country_code].groupby("year", as_index=False)["count"].sum()
    
    country = country.merge(right= country_population, how='left', on='year')
    country.drop(country[country['population'].isna()].index, inplace=True)
    country["percentage_of_population_migration"] = (country["count"] / country["population"]) * 100

    country.rename(columns={'count': "displaced"}, inplace=True)

    return country



In [7]:
venezuela = Country_population_data("VEN", population, asylum)
venezuela

,year,displaced,population,percentage_of_population_migration
0,1973,5,12463537.0,0.000040
1,1984,10,17008841.0,0.000059
2,1986,5,17923536.0,0.000028
3,1989,61,19348921.0,0.000315
4,1990,171,19827010.0,0.000862
5,1991,246,20304896.0,0.001212
6,1992,290,20780948.0,0.001396
7,1993,297,21254520.0,0.001397
8,1994,471,21725188.0,0.002168
9,1995,1084,22193821.0,0.004884


In [8]:
af = Country_population_data('AFG', population, asylum)
af.style.format(precision=4, thousands=".", decimal=",")


,year,displaced,population,percentage_of_population_migration
0,1.972,5,"11.853.696,0000","0,0000"
1,1.976,15,"13.059.851,0000","0,0001"
2,1.977,36,"13.340.756,0000","0,0003"
3,1.978,141,"13.611.441,0000","0,0010"
4,1.979,602,"13.655.567,0000","0,0044"
5,1.980,1.308.405,"13.169.311,0000","9,9353"
6,1.981,2.354.516,"11.937.581,0000","19,7236"
7,1.982,515.423,"10.991.378,0000","4,6893"
8,1.983,204.003,"10.917.982,0000","1,8685"
9,1.984,108.450,"11.190.221,0000","0,9691"


In [9]:
syria = Country_population_data('SYR', population, asylum)
syria

,year,displaced,population,percentage_of_population_migration
0,1970,176,6380609.0,0.002758
1,1971,209,6598094.0,0.003168
2,1972,143,6826024.0,0.002095
3,1973,5,7060321.0,0.000071
4,1974,5,7302160.0,0.000068
5,1975,10,7554364.0,0.000132
6,1976,247,7817320.0,0.003160
7,1977,254,8090501.0,0.003139
8,1978,373,8372759.0,0.004455
9,1979,84,8661909.0,0.000970


In [10]:
ukrine = Country_population_data("UKR", population, asylum)
ukrine.style.format(precision=4, thousands=".", decimal=",")

,year,displaced,population,percentage_of_population_migration
0,1.981,5,"50.189.281,0000","0,0000"
1,1.990,5,"52.054.092,0000","0,0000"
2,1.991,115,"52.170.961,0000","0,0002"
3,1.992,3.318,"52.323.671,0000","0,0063"
4,1.993,6.322,"52.350.126,0000","0,0121"
5,1.994,3.781,"52.075.998,0000","0,0073"
6,1.995,2.595,"51.665.539,0000","0,0050"
7,1.996,3.128,"51.226.109,0000","0,0061"
8,1.997,3.380,"50.786.885,0000","0,0067"
9,1.998,2.423,"50.379.731,0000","0,0048"


In [11]:
from plotly.subplots import make_subplots

fit = make_subplots(rows=1, cols=1)
fit.add_trace(go.Scatter(y=venezuela['percentage_of_population_migration'], x=venezuela["year"]))
fit.add_trace(go.Scatter(y=ukrine["percentage_of_population_migration"], x=ukrine["year"]))
fit.add_trace(go.Scatter(y=af["percentage_of_population_migration"], x=af["year"]))

fit.show()

# Adding population movement

In [12]:
asylum

,country_of_origin_abbr,country_of_origin_name,country_of_asylum_abbr,country_of_asylum_name,region_of_asylum,category,year,count
0,DZA,Algeria,MLT,Malta,Europe,Refugee,1962,5
1,AGO,Angola,COD,Dem. Rep. of the Congo,Southern Africa,Refugee,1962,20000
2,AGO,Angola,NAM,Namibia,Southern Africa,Refugee,1962,277
3,BDI,Burundi,NAM,Namibia,Southern Africa,Refugee,1962,13
4,CHN,China,NPL,Nepal,Asia and the Pacific,Refugee,1962,5
...,...,...,...,...,...,...,...,...
103338,ZWE,Zimbabwe,POL,Poland,Europe,Asylum-seekers,2024,9
103339,ZWE,Zimbabwe,ZAF,South Africa,Southern Africa,Asylum-seekers,2024,89
103340,ZWE,Zimbabwe,SWE,Sweden,Europe,Asylum-seekers,2024,5
103341,ZWE,Zimbabwe,USA,United States of America,Americas,Asylum-seekers,2024,641


In [13]:
def Country_by_destination(country_abbr: str, asy: pd.DataFrame) -> pd.DataFrame:
    country = asy[asy["country_of_origin_abbr"] == country_abbr]
    country_destination = country.groupby("country_of_asylum_name")["count"].sum().reset_index().sort_values(by="count", ascending=False)
    return country_destination

af_des = Country_by_destination("UKR", asylum)
af_des.head(20)

,country_of_asylum_name,count
68,Russian Federation,1797280
63,Poland,1273423
29,Germany,1232215
19,Czechia,586657
79,United Kingdom,249132
73,Spain,224870
38,Italy,194074
11,Bulgaria,187758
66,Rep. of Moldova,169543
67,Romania,168063


In [14]:
import plotly.express as px
# df = px.data.gapminder().query("year==2007")
# fig = px.scatter_geo(df, locations="iso_alpha", color="continent",
#                      hover_name="country", size="pop",
#                      projection="natural earth")
# fig.show()

In [15]:
px.scatter_mapbox(mapbox_style='open-street-map')

C:\Users\Euclides\AppData\Local\Temp\ipykernel_14188\3449835803.py:1: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



TypeError: Expected str or slice, got: <class 'NoneType'>

In [28]:
asylum["category"].unique()

array(['Refugee', 'Asylum-seekers', 'People in refugee-like situation',
       'Other people in need of international protection'], dtype=object)